In [1]:
%matplotlib inline

In [3]:
import numpy as np
import matplotlib.pylab as plt

## Simple/Toy Models

## Interrupts

### Time

Let $\lambda$ = arrival rate in requests/sec. On average, the time between two requests is:

$\delta t = \frac{1}{\lambda}$

We will analyze this fundamental interval of length $\delta t$. This interval can be broken into 3 chunks:

* $t_1$ = itr = $\tau$ ("tau")

* $t_2$ = time spent processing = $t_{\text{busy}}$

* $t_3$ = time spend sleeping

In other words, the claim is:

$\delta t = \frac{1}{\lambda} = t_1 + t_2 + t_3 = \tau + t_{\text{busy}} + t_3$

Let's posit the dependence of these terms on DVFS, $\Delta$ ("Delta").

Claim: $t_{\text{busy}} = A \frac{N_i}{\Delta^{1+\alpha}}$

Here, A is a constant of proportionality. We don't really care about it at this stage. It's something that can be inferred from the data (by "fitting").

$N_i$ = total number of instructions

$\Delta$ = DVFS which we are using as a proxy for CPU frequency

Naively, we would assume time is inversely proportional to frequency ($\alpha = 0$). $\alpha$ here is introduced to measure deviations from this naive behavior. This is also a free parameter or degree of freedom that is inferred from the data.

The last piece is $t_3$. This is the time remaining after waiting for $\tau$ (itr) and doing the processing. So, we would formally write this as:

$t_3 = \left[\frac{1}{\lambda} - t_{\text{busy}} - \tau \right]^{+}$

where $\left[x\right]^{+} = x$ if $x > 0$ and 0 otherwise. Read this as the positive part of $x$.

The two quantities we measure are latency and energy. Latency is:

$t_{\text{latency}} = \tau + t_{\text{busy}}$

We still need to discuss energy.

### Energy

The energy consumed in a certain time-period, t at a constant rate (power), P is simply E = Pt.

The total energy consumed in the fundamental time-interval, $\delta t$, is

$\text{E} = P_{\text{idle}} \tau + P_{\text{busy}} t_{\text{busy}} + P_{\text{sleep}} \left[\frac{1}{\lambda} - t_{\text{busy}} - \tau \right]^{+}$

Don't take the subscripts on the different power values too seriously. The point is that, potentially, each time-interval can consume energy at different rates.

We need to posit the DVFS dependence of the power values too. We know that CPU power, $\text{P} \propto V^2 f$ where V = voltage supplied and f = CPU frequency. DVFS affects both V and f and we are going to write this dependence as, $P_{\text{busy}} = B \Delta^{2+\beta}$. If the actual power goes as $\Delta^3$ for example, then $\beta = 1$. $\beta$ gives us another degree of freedom. For $P_{\text{idle}}$ and $P_{\text{sleep}}$, we will assume they are constants and not affected by DVFS. It is easy to put in DVFS dependence if we wanted to.

To simplify matters, assume that $t_3$ is positive.

$\delta t = \tau + t_{\text{busy}} + \left[\frac{1}{\lambda} - t_{\text{busy}} - \tau \right] = t_{\text{latency}} + \left[\frac{1}{\lambda} - t_{\text{busy}} - \tau \right]$


$\text{E} = P_{\text{idle}} \tau + P_{\text{busy}} t_{\text{busy}} + P_{\text{sleep}} \left[\frac{1}{\lambda} - t_{\text{busy}} - \tau \right]$




## Exponential Distributions and busy time vs interarrival time

We will assume that both busy time and interarrival times are exponentially distributed. A random variable $T$ is exponentially distributed


$$T \sim Exp(\lambda)$$

if it has the p.d.f. (probability distribution function):

$$p(t) = \lambda \exp(-\lambda t)$$

You can think of $p(t)$ as the probability that a time measurement falls between $t$ and $t+dt$ where $dt$ is an infinitesimal element. The leading $\lambda$ ensures normalization i.e. that the probabilities to observe any time is 1:

$$\int_0^{\infty} dt p(t) = 1$$

Our question is: Given two exponentially distributed random variables:

$$T_1 \sim Exp(\lambda_1)$$

and

$$T_2 \sim Exp(\lambda_2)$$

what is the probability that a measurement of $T_2$ is greater than that of $T_1$?

Let's first calculate that $T_2$ exceeds $T_1$ by an amount $t > 0$:

$$\mathbb{P}[T_2 - T_1 = t] = \int_0^{\infty}\int_0^{\infty} dt_1 dt_2 p_1(t_1) p_2(t_2) \delta(t_2 - t_1 - t)$$

where $p_1$ is the pdf of $T_1$, $p_2$ is the pdf of $T_2$ and $\delta$ is the Dirac-delta function that enforces the constraint. Doing the $t_2$ integral, we get

$$\mathbb{P}[T_2 - T_1 = t] = \int_0^{\infty} dt_1 \lambda_1 \exp(-\lambda_1 t_1) \lambda_2 \exp(-\lambda_2(t_1 + t)) $$

$$\mathbb{P}[T_2 - T_1 = t] = \lambda_1 \lambda_2 \exp(-\lambda_2 t) \int_0^{\infty} dt_1 \exp(-(\lambda_1+\lambda_2)t_1) = \frac{\lambda_1 \lambda_2}{\lambda_1 + \lambda_2} \exp(-\lambda_2 t)$$

Since we just care about $T_2 > T_1$, we need to integrate over all possible time-deltas $t$,

$$\mathbb{P}[T_2 > T_1] = \int_0^{\infty} dt \frac{\lambda_1 \lambda_2}{\lambda_1 + \lambda_2} \exp(-\lambda_2 t) 
= \boxed{\frac{\lambda_1}{\lambda_1 + \lambda_2}}$$


Also, it can be easily shown that the mean-time: $\mathbb{E}[T_i] = \frac{1}{\lambda_i}$ and since we measure mean times in our measurements, we can replace this by:

$$\mathbb{P}[T_2 > T_1] = \boxed{\frac{\frac{1}{\mathbb{E}[T_1]}}{\frac{1}{\mathbb{E}[T_1]} + \frac{1}{\mathbb{E}[T_2]}}}$$

In our case,

$T_1 = \text{ref cycles per iteration converted to time}$

$T_2 = \text{inter-arrival time}$
